<a href="https://colab.research.google.com/github/YoelCanaza/curso-transfer-learning-huggingface/blob/main/my_NLP_con_Hugging_Face.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NLP con Hugging Face

## Procesando los datos para NLP

### Descargando el dataset

In [1]:
%%capture
!pip install datasets transformers evaluate
!pip install transformers==4.30.2
!pip install accelerate==0.20.1
!pip install accelerate

Usaremos el dataset MRPC. Este es uno de los 10 datasets que componen el [benchmark (punto de referencia) GLUE](https://huggingface.co/datasets/glue). Se utiliza para medir el rendimiento de los modelos ML en 10 tareas de clasificación de texto diferentes.

Seleccionamos el subset `mrpc` del dataset `glue`:

In [2]:
from datasets import load_dataset

ds = load_dataset("glue", "mrpc") # indicamos el repo-id del dataset y el nombre del subset

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/3668 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/408 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1725 [00:00<?, ? examples/s]

Notemos que `mrpc` está compuesto de dos oraciones y una etiqueta que indica si los dos enunciados son equivalentes.

In [3]:
ex = ds["train"][19]
ex

{'sentence1': "The new Finder puts a user 's folders , hard drive , network servers , iDisk and removable media in one location , providing one-click access .",
 'sentence2': "Panther 's redesigned Finder navigation tool puts a user 's favourite folders , hard drive , network servers , iDisk and removable media in one location .",
 'label': 1,
 'idx': 21}

In [4]:
labels = ds["train"].features["label"]
labels

ClassLabel(names=['not_equivalent', 'equivalent'], id=None)

In [5]:
labels.int2str(0)

'not_equivalent'

### Tokenizando


Para vision computer pudimos descargar el feature extractor directamente del repositorio del modelo pre-entrenado que usamos como base.

Podemos pensar en la función tokenizadora como el equivalente en el NLP.

Vamos a procesar nuestros datos para que estén de la forma adecuada para ingresarlos a nuestro modelo.

Descargamos el tokenizador directamente del repo del modelo que usaremos. Demos utilizar el mismo tokenizador que se utilizó con el modleo preentrenado que usaremos para hacer el afinamiento posterior.

In [6]:
from transformers import AutoTokenizer # detectará cuál es el modelo y lo descargará

repo_id = "bert-base-uncased" # el modelo que queremos utilizar como preentrenado

tokenizer = AutoTokenizer.from_pretrained(repo_id)

# con esto descargaremos desde el hub de huggingface el tokenizer utilizado para el modelo bert-base-uncased


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

El tokenizer convierte texto en números legibles por el modelo. Números que capten o que representen cada palabra, cada parte, cada letra del texto que ingresamos.

A esos números los llamaremos los input ids

Para preprocesar el conjunto de datos necesitamos convertir el texto en números que el modelo pueda entender. Esto se hace con un tokenizador.

Pasar de texto a números se conoce como codificación o encoding. El encoding se realiza en un proceso de dos pasos: la tokenización, seguida de la conversión a input ids. Por el momento nos basta saber que estamos traduciendo texto a números llamados como input ids. Estos estarán en el formato adecuado para alimentar nuestro modelo.

Podemos alimentar al tokenizador con una oración o una lista de oraciones, por lo que podemos tokenizar directamente todas las primeras oraciones y todas las segundas oraciones de cada par de esta manera:

In [7]:
tokenized_sentence_1 = tokenizer(ds["train"]["sentence1"][5])
tokenized_sentence_1

{'input_ids': [101, 6599, 1999, 1996, 2034, 4284, 1997, 1996, 2095, 3333, 2321, 3867, 2013, 1996, 2168, 2558, 1037, 2095, 3041, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

input_ids = traducción de texto a números entendibles por el modelo

Necesitamos manejar los dos enunciados como un par y no separados. El tokenizador puede tomar un par de secuencias y prepararlas de la manera que espera nuestro modelo:

In [8]:
inputs = tokenizer("This is the first", "This is the second") # nos creará un diccionario con ambos enunciados a la vez
inputs

{'input_ids': [101, 2023, 2003, 1996, 2034, 102, 2023, 2003, 1996, 2117, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

¿Qué significa cada uno de los valores que nos retorna el tokenizador?
- `input_ids` es la traducción de palabras a números.
- `attention_mask` es un tensor con la misma forma que `input_ids`, pero lleno de 0 y 1: los 1 indican que se debe atender a los tokens correspondientes y los 0 indican que no se deben atender. Es decir, deben ser ignorados por el modelo.
- `token_type_ids` dice al modelo qué parte de la entrada es la primera oración y cuál es la segunda oración.

El modelo espera que las entradas sean de la forma [CLS] oración 1 [SEP] oración 2 [SEP] cuando hay dos oraciones.

In [9]:
tokenizer.convert_ids_to_tokens(inputs["input_ids"])

['[CLS]',
 'this',
 'is',
 'the',
 'first',
 '[SEP]',
 'this',
 'is',
 'the',
 'second',
 '[SEP]']

con CLS se indica que está iniciando todo nuestro ejemplo y con SEP estamos separando entre los dos enunciados. El segundo SEP indica que acabó el ejemplo

Si seleccionamos otro modelo en el Hub no necesariamente tendremos `token_type_ids` en las entradas tokenizadas (por ejemplo, no se devuelven si usa un modelo `DistilBERT`). Solo se devuelven cuando el modelo sabrá qué hacer con ellas, porque los ha visto durante su preentrenamiento.

En general, no necesitamos preocuparnos por si hay o no `token_type_ids` en nuestras entradas tokenizadas, siempre que usemos el tokenizador correspondiente al modelo, todo estará bien ya que el tokenizador sabe qué proporcionar al modelo.

Como ejemplo de esto utilizaremos un modelo [`distilroberta-base`](https://huggingface.co/distilroberta-base) por su tamaño y efectividad. Pero no cuenta con `token_type_ids` y aún así nos regresa excelentes resultados.


In [10]:
repo_id = "distilroberta-base"

tokenizer = AutoTokenizer.from_pretrained(repo_id)

config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Creamos una función tokenizadora. Recibe un ejemplo y lo tokeniza.

In [11]:
def tokenize_fn(example):
  return tokenizer(example["sentence1"], example["sentence2"], truncation=True) #truncation=True trunca el largo de las oraciones a uno determinado
  #es muy importante tener tensores cuadrados, un dataset cuadrado, todos los tensoress, todos los ejemplos deben ser de la misma longitud

In [12]:
# esta vez podemos usar la función map porque el texto es más sencillo de procesar que las imágenes
prepared_ds = ds.map(tokenize_fn, batched=True) # con map podemos aplicarle la función tokenize_fn a cada uno de los ejemplos

Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

### Definiendo el data collator: Dynamic padding

Necesitamos que nuestros tensores tengan una forma rectangular. Es decir que tengan el mismo tamaño cada uno de los ejemplos. Sin embargo, los textos no necesariamente tienen el mismo tamaño.

Para ello usamos el relleno o padding. El padding se asegura de que todas nuestras oraciones tengan la misma longitud al agregar una palabra especial llamada padding token a las oraciones con menos valores. Por ejemplo, si tenemos 10 oraciones con 10 palabras y 1 oración con 20 palabras, el relleno garantizará que todas las oraciones tengan 20 palabras.

Dejamos el argumento de `padding` del tokenizer vacío en nuestra función de tokenización por ahora. Esto se debe a que rellenar (hacer padding) todas las muestras hasta la longitud máxima del dataset no es eficiente, es mejor rellenar las muestras cuando estamos construyendo un batch, ya que entonces solo necesitamos rellenar hasta la longitud máxima en ese batch, y no la longitud máxima en todo el dataset. ¡Esto puede ahorrar mucho tiempo y potencia de procesamiento cuando las entradas tienen longitudes muy variables!

Usaremos un DataCollator para esto.

Rellenemos (hagamos padding) todos los ejemplos con la longitud del elemento más largo del batch. A esta técnica se le conoce como relleno dinámico o dynamic padding.

In [13]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

## Entrenamiento y evaluación

Definamos el resto de los argumentos necesarios para `Trainer`.

### Definiendo la métrica

In [14]:
import evaluate
import numpy as np

def compute_metrics(eval_pred): # recibe las predicciones del modelo
  metric = evaluate.load("glue", "mrpc")
  logits, labels = eval_pred # con esto recibimos una dupla de valores y la separamos en dos, una llamda logits y otra labels
  predictions = np.argmax(logits, axis=-1)
  return metric.compute(predictions=predictions, references=labels) # nos indicará qué porcentaje sí acierta nuestro modelo

### Configurando `Trainer`


In [15]:
from transformers import AutoModelForSequenceClassification
# con AutoModelForSequenceClassification vamos a poder importar nuestro modelo, pero con una última capa para clasificar secuencias

labels = ds["train"].features["label"].names

model = AutoModelForSequenceClassification.from_pretrained(
    repo_id,
    num_labels=len(labels),
    id2label={str(i): c for i, c in enumerate(labels)},
    label2id={c: str(i) for i, c in enumerate(labels)}
)

model.safetensors:   0%|          | 0.00/331M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'roberta.pooler.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at distilroberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias',

A nuestro modelo preetrenado le estamos cortando la cabeza para colocar una nueva cabeza para clasificación de texto con dos etiquetas

In [16]:
from transformers import TrainingArguments

training_args = TrainingArguments( # colocaremos todo lo que necesitamos para el entrenamiento
    output_dir = "./YoelCanaza/distilroberta-base-mrpc-glue-yoel-c",
    evaluation_strategy="steps",
    num_train_epochs=3,
    push_to_hub=True,
    load_best_model_at_end=True
)

In [17]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) y
Token is valid (permission: write).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' creden

In [18]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=prepared_ds["train"],
    eval_dataset=prepared_ds["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'Repository' (from 'huggingface_hub.repository') is deprecated and will be removed from version '1.0'. Please prefer the http-based alternatives instead. Given its large adoption in legacy code, the complete removal is only planned on next major release.
For more details, please read https://huggingface.co/docs/huggingface_hub/concepts/git_vs_http.
  warnings.warn(warning_message, FutureWarning)
Cloning https://huggingface.co/YoelCanaza/distilroberta-base-mrpc-glue-yoel-c into local empty directory.


### Entrenamiento

In [19]:
train_results = trainer.train()
trainer.save_model()
trainer.log_metrics("train", train_results.metrics)
trainer.save_metrics("train", train_results.metrics)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss,Accuracy,F1
500,0.514700,0.709730,0.821078,0.876481
1000,0.354200,0.640802,0.835784,0.877960


Upload file runs/Jan30_08-26-30_aa9b6b8fd0b4/events.out.tfevents.1706603334.aa9b6b8fd0b4.153.0: 100%|#########…

To https://huggingface.co/YoelCanaza/distilroberta-base-mrpc-glue-yoel-c
   724be97..2bb3c49  main -> main

   724be97..2bb3c49  main -> main

To https://huggingface.co/YoelCanaza/distilroberta-base-mrpc-glue-yoel-c
   2bb3c49..0278f2b  main -> main

   2bb3c49..0278f2b  main -> main



***** train metrics *****
  epoch                    =        3.0
  total_flos               =   191920GF
  train_loss               =     0.3728
  train_runtime            = 0:02:33.93
  train_samples_per_second =     71.484
  train_steps_per_second   =      8.945


### Evaluación

In [20]:
metrics = trainer.evaluate(prepared_ds["validation"])
trainer.log_metrics("eval", metrics)
trainer.save_metrics("eval", metrics)

***** eval metrics *****
  epoch                   =        3.0
  eval_accuracy           =     0.8358
  eval_f1                 =      0.878
  eval_loss               =     0.6408
  eval_runtime            = 0:00:01.67
  eval_samples_per_second =    243.561
  eval_steps_per_second   =     30.445


### Compartimos en el Hub

In [21]:
kwargs = {
    "finetuned_from": model.config._name_or_path,
    "tasks": "text-classification",
    "dataset": "datasetX",
    "tags": ["text-classification"]
}

trainer.push_to_hub(commit_message=" 🤗", **kwargs)

To https://huggingface.co/YoelCanaza/distilroberta-base-mrpc-glue-yoel-c
   0278f2b..0d917bb  main -> main

   0278f2b..0d917bb  main -> main

To https://huggingface.co/YoelCanaza/distilroberta-base-mrpc-glue-yoel-c
   0d917bb..820f1fd  main -> main

   0d917bb..820f1fd  main -> main



'https://huggingface.co/YoelCanaza/distilroberta-base-mrpc-glue-yoel-c/commit/0d917bb1b4e0508432ec81a1035c9c19104d3122'